In [ ]:
# Project 3 --- Dynamic Programming
# Om Patel
# op20@uakron.edu

In [ ]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [22]:
# Read a PGM file and return the image data, size, and max value
def readFile(fileName):
    lines = []
    with open(fileName, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip empty lines or comment lines
            if (not line or line.startswith('#')):
                continue
            lines.append(line)

    # Check the first line to be P2
    if (lines[0] != 'P2'):
        raise ValueError("Not a valid P2 file.")

    # Next line has xAxis and yAxis
    XY = lines[1].split()
    xAxis = int(XY[0])
    yAxis = int(XY[1])

    # Next line has maxValue
    maxValue = int(lines[2])

    # Remaining lines have the pixel data
    pixelValue = " ".join(lines[3:]).split()
    if (len(pixelValue) != xAxis * yAxis):
        raise ValueError("Pixel data size does not match image size.")

    image = []
    index = 0
    for _ in range(yAxis):
        row = []
        for _ in range(xAxis):
            row.append(int(pixelValue[index]))
            index += 1
        image.append(row)
    return image, xAxis, yAxis, maxValue

# Write the image data to a PGM file
def writeToFile(fileName, image, xAxis, yAxis, maxValue):
    with open(fileName, 'w') as file:
        file.write("P2\n")
        file.write(f"{xAxis} {yAxis}\n")
        file.write(f"{maxValue}\n")
        for row in image:
            file.write(" ".join(map(str, row)) + "\n")

# Calculate the energy of the image
def calculateEnergy(image, xAxis, yAxis):
    energy = [[0]*xAxis for _ in range(yAxis)]
    for j in range(yAxis):
        for i in range(xAxis):

            value = image[j][i]

            # for up neighbor
            if (j > 0):
                up = image[j-1][i]
            else:
                up = value

            # for down neighbor
            if (j < yAxis-1):
                down = image[j+1][i]
            else:
                down = value

            # for left neighbor
            if (i > 0):
                left = image[j][i-1]
            else:
                left = value

            # for right neighbor
            if (i < xAxis-1):
                right = image[j][i+1]
            else:   
                right = value

            # Calculate energy
            energy[j][i] = abs(value - up) + abs(value - down) + abs(value - left) + abs(value - right)

    return energy

# Calculate the cumulative energy
def calculateCumulativeVertical(energy, xAxis, yAxis):
    cumulativeEnergy = [[energy[r][c] for c in range(xAxis)] for r in range(yAxis)]
    for j in range(1, yAxis):
        for i in range(xAxis):
            minimumCumulativeEnergy = cumulativeEnergy[j-1][i]
            if i > 0:
                minimumCumulativeEnergy = min(minimumCumulativeEnergy, cumulativeEnergy[j-1][i-1])
            if i < xAxis-1:
                minimumCumulativeEnergy = min(minimumCumulativeEnergy, cumulativeEnergy[j-1][i+1])
            cumulativeEnergy[j][i] += minimumCumulativeEnergy
    return cumulativeEnergy

# Find the vertical seam with the least energy
def findVerticalSeam(cumulativeEnergy, xAxis, yAxis):
    seam = [-1]*yAxis
    # Find min in last row
    lastRow = cumulativeEnergy[yAxis-1]
    minimumIndex = 0
    minimumValue = lastRow[0]
    for i in range(1, xAxis):
        if (lastRow[i] < minimumValue):
            minimumValue = lastRow[i]
            minimumIndex = i
    seam[yAxis-1] = minimumIndex
    # Trace upwards
    for j in range(yAxis-2, -1, -1):
        i = seam[j+1]
        columnCandidates = [i]
        if (i > 0):
            columnCandidates.append(i-1)
        if (i < xAxis-1):
            columnCandidates.append(i+1)
        # pick left-most min from the row above
        minimumCumulativeEnergy = columnCandidates[0]
        for c in columnCandidates:
            if (cumulativeEnergy[j][c] < cumulativeEnergy[j][minimumCumulativeEnergy]):
                minimumCumulativeEnergy = c
        seam[j] = minimumCumulativeEnergy
    return seam

# Remove a vertical seam from the image
def removeSeam(image, seam, xAxis, yAxis):
    for j in range(yAxis):
        i = seam[j]
        image[j].pop(i)
    return image

# Remove n number of vertical seams from the image
def removeVerticalSeam(image, xAxis, yAxis, n):
    for _ in range(n):
        energy = calculateEnergy(image, xAxis, yAxis)
        cumulativeEnergy = calculateCumulativeVertical(energy, xAxis, yAxis)
        seam = findVerticalSeam(cumulativeEnergy, xAxis, yAxis)
        removeSeam(image, seam, xAxis, yAxis)
        xAxis -= 1
    return image, xAxis, yAxis

# Transpose function to switch x-axis and y-axis
def transpose(image):
    return list(map(list, zip(*image)))

# Remove n number of horizontal seams from the image
def removeHorizontalSeam(image, xAxis, yAxis, n):
    # Transpose the image
    transposeImage = transpose(image)

    # remove vertical seams from transposed image
    transposeImage, newY, newX = removeVerticalSeam(transposeImage, yAxis, xAxis, n)

    # Transpose back
    newImage = transpose(transposeImage)
    return newImage, newX, newY

In [ ]:
# test for seam curving with bug.pgm
# filename = "bug.pgm"
# verticalSeam2Remove = 10
# horizontalSeam2Remove = 8

# test for seam curving with CAS_2.pgm
# filename = "CAS_2.pgm"
# verticalSeam2Remove = 100
# horizontalSeam2Remove = 80

In [24]:
# Part I : Vertical seam removal only
# Save your processed file to img_processed_v_h.pgm
filename1 = filename.split(".")[0]+"_processed_"+str(verticalSeam2Remove)+"_0.pgm"
print(filename1)

# your code, add cells as you need
data, xAxis, yAxis, maxValue = readFile(filename)
data, xAxis, yAxis = removeVerticalSeam(data, xAxis, yAxis, verticalSeam2Remove)
writeToFile(filename1, data, xAxis, yAxis, maxValue)

CAS_2_processed_100_0.pgm


In [25]:
# Part II : both vertical and horizontal seams removal 
# Save your processed file to img_processed_v_h.pgm
filename2 = filename.split(".")[0]+"_processed_"+ \
            str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
print(filename2)

# your code, add cells as you need
data, xAxis, yAxis, maxValue = readFile(filename)  # reload original
data, xAxis, yAxis = removeVerticalSeam(data, xAxis, yAxis, verticalSeam2Remove)
data, xAxis, yAxis = removeHorizontalSeam(data, xAxis, yAxis, horizontalSeam2Remove)
writeToFile(filename2, data, xAxis, yAxis, maxValue)

CAS_2_processed_100_80.pgm


In [26]:
# This is for students taking Algorithms at graduate level (CPSC 535) 
filenameG = "flower.ppm"
vSeam2Remove = 20
hSeam2Remove = 25

In [27]:
# This is for students taking Algorithms at graduate level (CPSC 535) 
# Part III : color img in ppm
filenameG2 = filenameG.split(".")[0]+"_processed_"+ \
            str(vSeam2Remove)+"_"+str(hSeam2Remove)+".ppm"
print(filenameG2)
# your code, add cells as you need

flower_processed_20_25.ppm
